In [1]:
!git clone https://github.com/marco-c/missing_symbols.git

Cloning into 'missing_symbols'...
remote: Counting objects: 28, done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 28 (delta 4), reused 27 (delta 3), pack-reused 0
Unpacking objects: 100% (28/28), done.
Checking connectivity... done.


In [2]:
import os
from datetime import datetime, timedelta
from pyspark.sql import functions

In [3]:
known_modules = set([module[:-4].lower() for module in os.listdir('missing_symbols/known_modules')])

In [4]:
num_days = 1
days = [datetime.utcnow().date() - timedelta(1) - timedelta(i) for i in range(0, num_days)]

In [5]:
dataset = SQLContext(sc).read.load(['s3://telemetry-parquet/socorro_crash/v1/crash_date=' + day.strftime('%Y%m%d') for day in days], 'parquet')

In [6]:
modules = dataset\
.filter(dataset['product'] == 'Firefox')\
.select(functions.explode(dataset['json_dump']['modules']).alias('module')).rdd\
.map(lambda v: v['module'])\
.filter(lambda m: m['missing_symbols'])\
.filter(lambda m: m['filename'].lower() not in known_modules)\
.flatMap(lambda m: [((m['filename'], m['version']), 1)])\
.reduceByKey(lambda x, y: x + y)\
.map(lambda v: (v[0][0], [(v[0][1], v[1])]))\
.reduceByKey(lambda x, y: x + y)\
.sortBy(lambda v: sum(count for ver,count in v[1]), ascending=False)\
.collect()

In [7]:
len(modules)

9891

In [8]:
[(module, sum(count for ver, count in versions)) for module, versions in modules[:50]]

[(u'ntdll.dll', 91158),
 (u'KERNELBASE.dll', 53253),
 (u'kernel32.dll', 30999),
 (u'msmpeg2vdec.dll', 28964),
 (u'user32.dll', 27328),
 (u'combase.dll', 15126),
 (u'libsystem_kernel.dylib', 8754),
 (u'CoreFoundation', 4365),
 (u'odbcint.dll', 4184),
 (u'GdiPlus.dll', 4026),
 (u'HIToolbox', 3939),
 (u'libdispatch.dylib', 3851),
 (u'mf.dll', 3787),
 (u'rpcrt4.dll', 3638),
 (u'AppKit', 3590),
 (u'd3d11.dll', 3325),
 (u'xul.dll', 3165),
 (u'libsystem_pthread.dylib', 2994),
 (u'icudt56.dll', 2869),
 (u'l3codecx.ax', 2849),
 (u'd3d9.dll', 2831),
 (u'wininet.dll', 2742),
 (u'xpsp2res.dll', 2730),
 (u'EMET.dll', 2550),
 (u'icudt52.dll', 2394),
 (u'linux-gate.so', 1955),
 (u'NPSWF32_24_0_0_154.dll', 1868),
 (u'VideoWindow.dll', 1779),
 (u'DHSurveillanceDll.dll', 1771),
 (u'dhnetsdk.dll', 1751),
 (u'iertutil.dll', 1734),
 (u'msctf.dll', 1728),
 (u'VideoToolbox', 1698),
 (u'dhplay.dll', 1696),
 (u'shell32.dll', 1600),
 (u'prochook.dll', 1592),
 (u'aetpkss1.dll', 1483),
 (u'igdumd32.dll', 1408),
 